In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q facenet_pytorch

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image

import torch
from torch import optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, SubsetRandomSampler, Dataset

import torchvision
from torchvision import transforms, datasets
from torchvision.datasets import ImageFolder
from torchvision.transforms import Resize

from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training

import matplotlib.pyplot as plt

In [ ]:
!pip show facenet-pytorch

Name: facenet-pytorch
Version: 2.5.2
Summary: Pretrained Pytorch face detection and recognition models
Home-page: https://github.com/timesler/facenet-pytorch
Author: Tim Esler
Author-email: tim.esler@gmail.com
License: UNKNOWN
Location: /usr/local/lib/python3.8/dist-packages
Requires: numpy, pillow, requests, torchvision
Required-by: 


In [ ]:
ON_COLAB = 'google.colab' in str(get_ipython())
RANDOM_SEED = 310123
BATCH_SIZE = 256 if torch.cuda.is_available() else 64

EPOCHS = 10
LEARNING_RATE = 1e-3

WORKERS = int(os.cpu_count() / 2)
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

if ON_COLAB:
    print("Running on Google Colab")
    DARK_UNDERSAMPLED_PATH = '/content/drive/MyDrive/xai_faces/dark_undersampled' # '/content/drive/MyDrive/xai_faces/dark_undersampled_abridged_cropped'
    LIGHT_UNDERSAMPLED_PATH = '/content/drive/MyDrive/xai_faces/light_undersampled' # '/content/drive/MyDrive/xai_faces/light_undersampled_abridged_cropped'
else:
    print("Not running on Google Colab")
    DARK_UNDERSAMPLED_PATH = '../data/dark_undersampled' # 'data/dark_undersampled_abridged_cropped'
    LIGHT_UNDERSAMPLED_PATH = '../data/light_undersampled' # 'data/light_undersampled_abridged_cropped/'

Running on Google Colab


## Preprocessing the Images

Cropping

### Light Undersampled

In [ ]:
# help(MTCNN)

# for preprocessing the images
# help(MTCNN)
mtcnn = MTCNN(image_size = 256, margin=0, min_face_size = 20, thresholds = [0.6, 0.7, 0.7], factor = 0.709, post_process = True, select_largest = True, device = DEVICE) # image_size = 160


In [ ]:
dataset = ImageFolder(LIGHT_UNDERSAMPLED_PATH, transform = Resize((512, 512)))

dataset.samples = [
    (p, p.replace(LIGHT_UNDERSAMPLED_PATH, LIGHT_UNDERSAMPLED_PATH + '_cropped'))
        for p, _ in dataset.samples
]

loader = DataLoader(
    dataset,
    num_workers = WORKERS,
    batch_size = BATCH_SIZE,
    collate_fn = training.collate_pil
)

In [ ]:
from tqdm import tqdm

In [ ]:
for i, (x, y) in tqdm(enumerate(loader), total = len(loader)):
        mtcnn(x, save_path = y)

  0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/facenet_pytorch/models/utils/detect_face.py:183: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  batch_boxes, batch_points = np.array(batch_boxes), np.array(batch_points)
/usr/local/lib/python3.8/dist-packages/facenet_pytorch/models/mtcnn.py:339: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  boxes = np.array(boxes)
/usr/local/lib/python3.8/dist-packages/facenet_pytorch/models/mtcnn.py:340: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a 

### Dark Undersampled

In [ ]:
dataset = ImageFolder(DARK_UNDERSAMPLED_PATH, transform = Resize((512, 512)))

dataset.samples = [
    (p, p.replace(DARK_UNDERSAMPLED_PATH, DARK_UNDERSAMPLED_PATH + '_cropped'))
        for p, _ in dataset.samples
]

loader = DataLoader(
    dataset,
    num_workers = WORKERS,
    batch_size = BATCH_SIZE,
    collate_fn = training.collate_pil
)

In [ ]:
# prevent warnings from cropping images
with np.testing.suppress_warnings() as sup:
    sup.filter(category = np.VisibleDeprecationWarning)
    for i, (x, y) in tqdm(enumerate(loader), total = len(loader)):
        mtcnn(x, save_path = y)

100%|██████████| 31/31 [10:13<00:00, 19.80s/it]


## Full

In [ ]:
from tqdm import tqdm

In [ ]:
FULL_PATH = '/content/drive/MyDrive/xai_faces/diverse_human_faces'

dataset = ImageFolder(FULL_PATH, transform = Resize((512, 512)))

dataset.samples = [
    (p, p.replace(FULL_PATH, FULL_PATH + '_cropped'))
        for p, _ in dataset.samples
]

loader = DataLoader(
    dataset,
    num_workers = WORKERS,
    batch_size = BATCH_SIZE,
    collate_fn = training.collate_pil
)

In [ ]:
for i, (x, y) in tqdm(enumerate(loader), total = len(loader)):
        mtcnn(x, save_path = y)

  0%|          | 0/107 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/facenet_pytorch/models/utils/detect_face.py:183: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  batch_boxes, batch_points = np.array(batch_boxes), np.array(batch_points)
/usr/local/lib/python3.8/dist-packages/facenet_pytorch/models/mtcnn.py:339: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  boxes = np.array(boxes)
/usr/local/lib/python3.8/dist-packages/facenet_pytorch/models/mtcnn.py:340: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a

In [ ]:
mtcnn = MTCNN(image_size = 256, margin=0, min_face_size = 1, thresholds = [0.1], factor = 0.709, post_process = True, select_largest = True, device = DEVICE)

In [ ]:
from tqdm import tqdm

In [ ]:
!ls /content/drive/MyDrive/xai_faces/

In [ ]:
MISSING_PATH = '/content/drive/MyDrive/xai_faces/missing_images'

dataset = ImageFolder(MISSING_PATH, transform = Resize((512, 512)))

dataset.samples = [
    (p, p.replace(MISSING_PATH, MISSING_PATH + '_cropped'))
        for p, _ in dataset.samples
]

loader = DataLoader(
    dataset,
    num_workers = WORKERS,
    batch_size = BATCH_SIZE,
    collate_fn = training.collate_pil
)

with np.testing.suppress_warnings() as sup:
    sup.filter(category = np.VisibleDeprecationWarning)
    for i, (x, y) in tqdm(enumerate(loader), total = len(loader)):
        mtcnn(x, save_path = y)

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
with np.testing.suppress_warnings() as sup:
    sup.filter(category = np.VisibleDeprecationWarning)
    for i, (x, y) in tqdm(enumerate(loader), total = len(loader)):
        mtcnn(x, save_path = y)